In [ ]:
from google.colab import files

uploaded = files.upload()

Saving income_district.csv to income_district.csv
Saving h27_work_status_df.csv to h27_work_status_df.csv
Saving h27_marriage_df.csv to h27_marriage_df.csv
Saving h27_labor_df.csv to h27_labor_df.csv
Saving h27_job_df.csv to h27_job_df.csv
Saving h27_indusry_df.csv to h27_indusry_df.csv
Saving h27_house_info_df.csv to h27_house_info_df.csv
Saving h27_house_df.csv to h27_house_df.csv
Saving h27_gender_df2.csv to h27_gender_df2.csv
Saving h27_family_df.csv to h27_family_df.csv
Saving h27_age_df.csv to h27_age_df.csv
Saving DS_Store to DS_Store


In [ ]:
import pandas as pd

income = pd.read_csv('income_district.csv')
work = pd.read_csv('h27_work_status_df.csv')
marriage = pd.read_csv('h27_marriage_df.csv')
labor = pd.read_csv('h27_labor_df.csv')
job = pd.read_csv('h27_job_df.csv')
industry = pd.read_csv('h27_indusry_df.csv')
house_info = pd.read_csv('h27_house_info_df.csv')
house = pd.read_csv('h27_house_df.csv')
gender = pd.read_csv('h27_gender_df2.csv')
family = pd.read_csv('h27_family_df.csv')
age = pd.read_csv('h27_age_df.csv')

print(income.head())
print(work.head())
print(marriage.head())
print(labor.head())
print(job.head())
print(industry.head())
print(house_info.head())
print(house.head())
print(gender.head())
print(family.head())
print(age.head())


   area_code  income_mean
0      13228   480.984419
1      43514   439.775281
2      23237   487.507769
3      18208   505.495103
4      12238   385.423846
   district_id  district2_id  level_identifier state_name city_name  \
0        37201           NaN                 1        香川県       高松市   
1        37201          10.0                 2        香川県       高松市   
2        37201          20.0                 2        香川県       高松市   
3        37201          30.0                 2        香川県       高松市   
4        37201        3000.0                 3        香川県       高松市   

  district_name district2_name employer_male self_employed_male  \
0           NaN            NaN         91038              11731   
1           塩屋町            NaN            30                  7   
2           築地町            NaN           144                 25   
3           塩上町            NaN           284                 53   
4           塩上町            NaN            14                  5   

  family_work_

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer # 追加
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

print("--- 実際のデータファイルの読み込みと結合を開始します ---")

# ファイル名リスト
income_file = 'income_district.csv'
census_files = [
    'h27_work_status_df.csv',
    'h27_marriage_df.csv',
    'h27_labor_df.csv',
    'h27_job_df.csv',
    'h27_indusry_df.csv',
    'h27_house_info_df.csv',
    'h27_house_df.csv',
    'h27_gender_df2.csv',
    'h27_family_df.csv',
    'h27_age_df.csv'
]

dataframes = {}

# 収入データを読み込む
try:
    income_df = pd.read_csv(income_file)
    # 'area_code' を 'KEY_CODE' に、'income_mean' を 'Income' に
    if 'area_code' in income_df.columns and 'income_mean' in income_df.columns:
        income_df = income_df.rename(columns={'area_code': 'KEY_CODE', 'income_mean': 'Income'})
    else:
        print(f"エラー: {income_file} に 'area_code' または 'income_mean' 列が見つかりません。")
        raise ValueError(f"'area_code' or 'income_mean' column not found in {income_file}")

    dataframes['income_district'] = income_df[['KEY_CODE', 'Income']].copy()
    # income_district の KEY_CODE を文字列型に
    dataframes['income_district']['KEY_CODE'] = dataframes['income_district']['KEY_CODE'].astype(str)

    print(f"\n- {income_file} が正常に読み込まれました。")
    print(dataframes['income_district'].head())
    print(f"  {income_file} の KEY_CODE のデータ型: {dataframes['income_district']['KEY_CODE'].dtype}") # KEY_CODEのデータ型を確認
    print(f"  {income_file} の KEY_CODE のユニーク数: {dataframes['income_district']['KEY_CODE'].nunique()}") # KEY_CODEのユニーク数を確認
    print(f"  {income_file} の KEY_CODE の最初の5つ: {dataframes['income_district']['KEY_CODE'].head().tolist()}") # KEY_CODEの最初の5つを確認
except FileNotFoundError:
    print(f"エラー: {income_file} が見つかりません。ファイルパスを確認してください。")
    exit() # ファイルが見つからない場合はスクリプトを終了
except Exception as e:
    print(f"エラー: {income_file} の読み込みまたは処理中に問題が発生しました: {e}")
    exit()

# 国勢調査関連データを読み込む
for f in census_files:
    try:
        df = pd.read_csv(f)
        df_name = f.replace('.csv', '')

        # level_identifier が '1' の行（市区町村全体）のみを対象とする
        if 'district_id' in df.columns and 'level_identifier' in df.columns:
            df_filtered = df[df['level_identifier'] == 1].copy() # level_identifierが1の行のみを抽出
            if df_filtered.empty:
                print(f"警告: {f} に level_identifier=1 の行が見つかりません。このファイルはスキップします。")
                continue
            df_filtered['KEY_CODE'] = df_filtered['district_id'].astype(str) # district_idをKEY_CODEとする

        if 'KEY_CODE' not in df_filtered.columns: # フィルタリング後もKEY_CODEがない場合
            print(f"警告: {f} に適切な結合キー ('district_id' と level_identifier=1 の行) が見つかりません。このファイルはスキップします。")
            continue

        # 重複する可能性のあるID/名称列を除外し、特徴量として利用する列を抽出
        cols_to_keep = ['KEY_CODE']
        for col in df_filtered.columns:
            # 結合キーや不要なID/名称列は除外
            if col not in ['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name', 'KEY_CODE']:
                # 'X' や '-' などの非数値データをNaNに変換してから、数値型に変換
                df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')
                cols_to_keep.append(col)

        dataframes[df_name] = df_filtered[cols_to_keep].copy()
        print(f"- {f} が正常に読み込まれました。")
        print(f"  {f} の KEY_CODE のデータ型: {dataframes[df_name]['KEY_CODE'].dtype}")
        print(f"  {f} の KEY_CODE のユニーク数: {dataframes[df_name]['KEY_CODE'].nunique()}")
        print(f"  {f} の KEY_CODE の最初の5つ: {dataframes[df_name]['KEY_CODE'].head().tolist()}")

    except FileNotFoundError:
        print(f"エラー: {f} が見つかりません。このファイルはスキップします。")
    except Exception as e:
        print(f"エラー: {f} の読み込みまたは処理中に問題が発生しました: {e}")

# データフレームの結合
# income_dfをベースに、他のデータフレームを順番に結合
if 'income_district' not in dataframes:
    print("エラー: 収入データが読み込まれていないため、結合を開始できません。")
    exit()

merged_df = dataframes['income_district'].copy() # 最初のmerged_dfはincome_districtから開始
print("\n--- 全てのデータフレームを結合中 ---")
print(f"初期 merged_df の行数: {merged_df.shape[0]}")
print(f"初期 merged_df の KEY_CODE のデータ型: {merged_df['KEY_CODE'].dtype}")

for df_name, df_to_merge in dataframes.items():
    if df_name == 'income_district':
        continue

    print(f"\n  結合対象: {df_name}")
    print(f"  {df_name} の KEY_CODE のデータ型: {df_to_merge['KEY_CODE'].dtype}")
    print(f"  {df_name} の KEY_CODE のユニーク数: {df_to_merge['KEY_CODE'].nunique()}")
    print(f"  {df_name} の KEY_CODE の最初の5つ: {df_to_merge['KEY_CODE'].head().tolist()}")

    # 結合するデータフレームから 'KEY_CODE' 以外の重複列を除外
    common_cols_except_key = [col for col in merged_df.columns if col in df_to_merge.columns and col != 'KEY_CODE']
    df_to_merge_clean = df_to_merge.drop(columns=common_cols_except_key, errors='ignore')

    initial_rows = merged_df.shape[0]
    merged_df = pd.merge(merged_df, df_to_merge_clean, on='KEY_CODE', how='inner')

    print(f"  - {df_name} を結合しました。結合前の行数: {initial_rows} -> 結合後の行数: {merged_df.shape[0]} 行")
    if merged_df.shape[0] == 0:
        print(f"警告: {df_name} との結合によりデータフレームが空になりました。KEY_CODEの不一致が原因の可能性があります。")
        # 結合をここで中断し、原因究明を促す
        break # ループを抜ける

if merged_df.shape[0] == 0:
    print("\nエラー: 結合処理の結果、データフレームが空になりました。")
    print("KEY_CODEの不一致、または結合条件が厳しすぎることが考えられます。")
    print("各ファイルの 'KEY_CODE' の内容と形式（例: '123' と '123_0'）を再確認し、必要であればデータクリーニングや結合方法（inner/left/outerなど）の見直しを検討してください。")
    exit() # データフレームが空の場合は以降の処理を停止

print("\n--- 全データフレームの結合が完了しました ---")
print("\n結合後のデータフレームの最初の5行:")
print(merged_df.head())
print("\n結合後のデータフレームの基本情報:")
merged_df.info()

# 欠損値の確認
print("\n--- 欠損値の確認 ---")
missing_values = merged_df.isnull().sum()
print(missing_values[missing_values > 0]) # 欠損値がある列のみ表示

# データ型の確認
print("\n--- データ型の確認 ---")
print(f"数値列のデータ型: {merged_df.select_dtypes(include=np.number).dtypes.tolist()}")
print(f"カテゴリカル列のデータ型: {merged_df.select_dtypes(include='object').dtypes.tolist()}")

# 'Income' 列が数値型でない場合は変換を試みる
if merged_df['Income'].dtype == 'object':
    print("\n'Income' 列がオブジェクト型です。数値型に変換を試みます。")
    merged_df['Income'] = pd.to_numeric(merged_df['Income'], errors='coerce')
    # 変換後に欠損値が発生した場合の処理
    if merged_df['Income'].isnull().any():
        print("数値変換後に'Income'列に欠損値が発生しました。これらを削除します。")
        merged_df.dropna(subset=['Income'], inplace=True)
elif not np.issubdtype(merged_df['Income'].dtype, np.number):
    # Income列がobject型でなくても、数値型でない場合 (例: mixed types)
    print("\n'Income' 列が予期せぬデータ型です。数値型に変換を試みます。")
    merged_df['Income'] = pd.to_numeric(merged_df['Income'], errors='coerce')
    if merged_df['Income'].isnull().any():
        print("数値変換後に'Income'列に欠損値が発生しました。これらを削除します。")
        merged_df.dropna(subset=['Income'], inplace=True)

# --- データ前処理、可視化、機械学習モデルの構築と評価 ---
print("\n--- データ前処理、可視化、機械学習モデルの構築と評価を開始します ---")

# 特徴量 (X) とターゲット変数 (y) の特定
features = [col for col in merged_df.columns if col not in ['KEY_CODE', 'Income']]
target = 'Income'

X = merged_df[features]
y = merged_df[target]

# データフレームが空でないことを確認してから train_test_split を実行
if X.empty or y.empty:
    print("\nエラー: 特徴量またはターゲット変数のデータフレームが空のため、モデル訓練をスキップします。")
    print("結合処理のログを確認し、データフレームが空になった原因を特定してください。")
    exit()

# --- データの分割 ---
print("\n--- データの分割 (訓練データとテストデータ) ---")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"訓練データ数: {len(X_train)}")
print(f"テストデータ数: {len(X_test)}")

# --- 前処理パイプラインの構築 ---
print("\n--- 前処理パイプラインの構築 ---")

# 数値列とカテゴリカル列を特定
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

# 数値特徴量にImputerを追加
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')), # 中央値で補完
            ('scaler', StandardScaler())
        ]), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

print(f"数値特徴量: {numerical_features}")
print(f"カテゴリカル特徴量: {categorical_features}")
print("前処理器が構築されました。数値特徴量には中央値補完と標準化が適用されます。")

# --- 可視化 ---
print("\n--- データの可視化 ---")

# ターゲット変数の分布
plt.figure(figsize=(8, 6))
sns.histplot(merged_df['Income'], kde=True, bins=20)
plt.title('収入の分布')
plt.xlabel('収入')
plt.ylabel('度数')
plt.savefig('income_distribution.png')
print("「income_distribution.png」を作成しました。")
plt.close()

# 収入と人口の散布図 (もし 'population' 列がある場合)
if 'population' in merged_df.columns:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='population', y='Income', data=merged_df)
    plt.title('収入と人口の関係')
    plt.xlabel('人口')
    plt.ylabel('収入')
    plt.savefig('income_vs_population.png')
    print("「income_vs_population.png」を作成しました。")
    plt.close()
else:
    print("警告: 'population' 列が見つからないため、収入と人口の散布図は生成できませんでした。")

# --- モデルの定義と訓練、評価 ---
print("\n--- モデルの定義、訓練、評価 ---")

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n--- {name} の訓練と評価 ---")
    # パイプラインの構築
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])

    # モデルの訓練
    pipeline.fit(X_train, y_train)

    # テストデータでの予測
    y_pred = pipeline.predict(X_test)

    # 評価指標の計算
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    results[name] = {'MSE': mse, 'RMSE': rmse, 'R2': r2}

    print(f"  MSE: {mse:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R^2: {r2:.4f}")

print("\n--- 全モデルの評価結果 ---")
for name, metrics in results.items():
    print(f"\nモデル: {name}")
    print(f"  MSE: {metrics['MSE']:.2f}")
    print(f"  RMSE: {metrics['RMSE']:.2f}")
    print(f"  R^2: {metrics['R2']:.4f}")

print("\nモデルの構築、訓練、評価が完了しました。")

--- 実際のデータファイルの読み込みと結合を開始します ---
エラー: income_district.csv が見つかりません。ファイルパスを確認してください。
エラー: h27_work_status_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_marriage_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_labor_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_job_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_indusry_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_house_info_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_house_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_gender_df2.csv が見つかりません。このファイルはスキップします。
エラー: h27_family_df.csv が見つかりません。このファイルはスキップします。
エラー: h27_age_df.csv が見つかりません。このファイルはスキップします。
エラー: 収入データが読み込まれていないため、結合を開始できません。


KeyError: 'income_district'